In [8]:
import pandas as pd
from google.cloud import storage
from google.colab import auth
from dbfread import DBF
import logging

def processar_arquivo_dbf(local_file_path, gcs_bucket_name, novo_nome, encoding='utf-8'):
    # Inicializar o cliente do Google Cloud Storage
    storage_client = storage.Client()

    try:
        # Baixar o arquivo DBF localmente
        bucket = storage_client.bucket(gcs_bucket_name)
        blob = bucket.blob(f"DBF/{novo_nome}")
        blob.download_to_filename(local_file_path)

        # Lista para armazenar os registros do DBF
        records = []

        # Ler o arquivo DBF e adicionar os registros à lista
        with DBF(local_file_path, encoding=encoding) as table:
            for record in table:
                records.append(record)

        # Converter a lista de registros em um DataFrame pandas
        df = pd.DataFrame(records)

        # Caminho para salvar o arquivo CSV
        csv_file_path = f"/tmp/{novo_nome.replace('.dbf', '.csv')}"

        # Salvar o DataFrame como um arquivo CSV
        df.to_csv(csv_file_path, index=False)

        print(f"Arquivo CSV salvo localmente em: {csv_file_path}")

        # Upload do arquivo CSV para o Google Cloud Storage
        gcs_blob_name = f"tratado/dimensao/{novo_nome.replace('.dbf', '.csv')}"
        blob = bucket.blob(gcs_blob_name)
        blob.upload_from_filename(csv_file_path)

        print(f"Arquivo CSV enviado com sucesso para o Google Cloud Storage: gs://{gcs_bucket_name}/{gcs_blob_name}")

    except FileNotFoundError as e:
        logging.error(f"O arquivo {novo_nome} não foi encontrado: {e}")
    except Exception as e:
        logging.error(f"Erro ao processar o arquivo {novo_nome}: {e}")

# Configurar o registro de logs
logging.basicConfig(filename='processamento_dbf.log', level=logging.ERROR)

# Autenticação do usuário
auth.authenticate_user()

# Lista de Ufs, Anos e Meses
ufs = ['CBO']

# Caminho do bucket do Google Cloud Storage
gcs_bucket_name = "tabsus"

# Renomear e processar os arquivos DBF
for uf in ufs:
    # Novo nome do arquivo
    novo_nome = f"{uf}.dbf"

    # Caminho do arquivo local
    local_file_path = f"/tmp/{novo_nome}"

    # Processar o arquivo DBF
    processar_arquivo_dbf(local_file_path, gcs_bucket_name, novo_nome, encoding='latin1')  # Tente usar 'latin1' como codificação


Arquivo CSV salvo localmente em: /tmp/CBO.csv
Arquivo CSV enviado com sucesso para o Google Cloud Storage: gs://tabsus/tratado/dimensao/CBO.csv
